Nama : Dary Ramadhan Abdussalam

# Business Understanding
## Latar Belakang dan Tantangan yang Dihadapi

Perusahaan atau organisasi sering kali memiliki banyak dokumen dalam format PDF yang berisi informasi penting, seperti laporan tahunan, dokumentasi produk, riset pasar, dan lainnya. Mengelola dan mengekstraksi informasi dari dokumen-dokumen ini bisa menjadi tugas yang sangat melelahkan dan memakan waktu. Tantangan utama yang dihadapi adalah:

1. Volume Dokumen yang Besar: Banyaknya dokumen yang perlu diakses dan diproses secara manual sangat memakan waktu.
2. Akses Informasi: Mengidentifikasi dan mengekstraksi informasi yang relevan dari dokumen besar atau banyak dokumen bisa sulit dan tidak efisien.
3. Kebutuhan Beragam Pengguna: Berbagai divisi dalam perusahaan mungkin memerlukan informasi yang berbeda dari dokumen yang sama
4. Bahasa Dokumen: Dokumen bisa tersedia dalam berbagai bahasa, sehingga menambah kompleksitas dalam mengekstraksi dan meringkas informasi.

## Tujuan dan Solusi yang Diusulkan
Tujuan utama dari proyek ini adalah untuk mengembangkan alat yang mampu membantu dalam mengekstraksi dan meringkas informasi dari dokumen PDF dengan cepat dan efisien. Solusi yang diusulkan adalah sistem interaktif berbasis yang dapat melakukan:

1. Question and Answer (Q&A): Pengguna dapat mengajukan pertanyaan spesifik terkait konten dokumen dan mendapatkan jawaban langsung.
2. Summarization: Pengguna dapat memperoleh ringkasan dari dokumen atau bagian tertentu dari dokumen dalam bahasa pilihan mereka (Inggris atau Indonesia).
   
Dengan sistem ini, diharapkan perusahaan dapat meningkatkan efisiensi dalam mengakses dan mengelola informasi penting dari dokumen PDF.

# Import Library

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.summarize import load_summarize_chain
import textwrap
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Data Preparation

## Memuat File .env dan Membuat Objek LLM

In [ ]:
# load file .env secara otomatis
load_dotenv()

True

In [ ]:
# membuat objek LLM menggunakan Gemini-pro
gemini_10_pro = GoogleGenerativeAI(model = "gemini-pro")

Pertama, kode load_dotenv() digunakan untuk memuat variabel lingkungan dari file .env secara otomatis, yang berguna untuk menyimpan informasi sensitif seperti kunci API. Kemudian, objek LLM (Language Model) dibuat menggunakan model gemini-pro dari Google Generative AI dengan kode gemini_10_pro = GoogleGenerativeAI(model = "gemini-pro").

## Memproses Dokumen PDF

In [ ]:
pdf_loader = PyPDFLoader('data_input/About_Bart.pdf', extract_images=True)
pdf_data = pdf_loader.load()

In [ ]:
type(pdf_data)

list

In [ ]:
len(pdf_data)

10

Dokumen PDF dimuat menggunakan pdf_loader = PyPDFLoader('data_input/About_Bart.pdf', extract_images=True), yang membuat objek loader untuk memuat file PDF About_Bart.pdf dari direktori data_input dan mengekstrak gambar jika ada. Data dari PDF tersebut kemudian diambil dan disimpan dalam variabel pdf_data dengan pdf_data = pdf_loader.load(). Tipe data dan jumlah halaman atau elemen dalam data PDF tersebut diperiksa menggunakan type(pdf_data) dan len(pdf_data).

## Membagi Dokumen Menjadi Potongan-potongan Lebih Kecil

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 450,
    separators=[
                "\n\n",
                "\n",
                " ",
                ".",
                ",",
                "\u200b",  # zero-width space
                "\uff0c",  # full-width comma
                "\u3001",  # ideographic comma
                "\uff0e",  # full-width full stop
                "\u3002",  # ideographic full stop
                "",
            ])

splits = text_splitter.split_documents(pdf_data)

In [ ]:
type(splits)

list

In [ ]:
len(splits)

76

Untuk membagi dokumen menjadi potongan-potongan teks yang lebih kecil, dibuat objek text_splitter menggunakan RecursiveCharacterTextSplitter, yang diatur dengan ukuran 1000 karakter dan tumpang tindih sebesar 450 karakter, serta menggunakan berbagai jenis spasi dan tanda baca sebagai pemisah. Dokumen PDF yang sudah dimuat kemudian dibagi menjadi beberapa potongan teks berdasarkan pengaturan ini dengan splits = text_splitter.split_documents(pdf_data). Tipe data dan jumlah potongan teks yang dihasilkan diperiksa dengan type(splits) dan len(splits).

## Membuat dan Memuat Vectorstore

In [ ]:
def create_vectorstore_folder(documents, embedding, persist_directory):
    vectorstore = Chroma.from_documents(
        documents = documents,
        embedding = embedding,
        persist_directory = persist_directory
    )

    return vectorstore

In [ ]:
# vectorstore_gemini = create_vectorstore_folder(
#     documents = splits,
#     embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
#     persist_directory = 'data_input/embedding_data'
# )

In [ ]:
# memanggil embedding dari directory yang sudah disimpan
vec_gemini = Chroma(persist_directory = 'data_input/embedding_data',
                    embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

Fungsi create_vectorstore_folder dibuat untuk membuat dan menyimpan vectorstore yang berisi embedding dari dokumen yang telah dibagi, menggunakan Chroma.from_documents. Vectorstore tersebut disimpan di direktori yang ditentukan. Meskipun pembuatan vectorstore di bagian ini dikomentari, vectorstore yang sudah disimpan kemudian dipanggil dengan vec_gemini = Chroma(persist_directory = 'data_input/embedding_data', embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001")).

## Membuat Template Q&A dan Chain

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
qa_template = """
    Anda adalah asisten yang hebat dalam memahami konteks tambahan.

    Gunakan potongan konteks berikut untuk menjawab pertanyaan di akhir.
    Gunakan minimal tiga kalimat untuk menjawab pertanyaan tersebut.
    Usahakan untuk menjawab sekomplet mungkin dengan gaya bahasa yang mudah dimengerti.
    Selalu akhiri jawaban dengan "terima kasih atas pertanyaannya!".

    {context}

    Pertanyaan: {question}

    Jawaban yang Bermanfaat:"""

custom_rag_prompt = PromptTemplate.from_template(qa_template)


In [ ]:
def create_qa_chain(retriever, llm):

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | custom_rag_prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

In [ ]:
gemini_chain = create_qa_chain(retriever = vec_gemini.as_retriever(), llm = ChatGoogleGenerativeAI(model = "gemini-pro"))

## Menampilkan Output Q&A

In [ ]:
#Testing
print(
    textwrap.fill(
        gemini_chain.invoke("Apa itu BART")  ,
        width=100
    )
)

BART (Bidirectional Auto-Regressive Transformers) adalah model bahasa yang menggabungkan pemahaman
dan pembangkitan bahasa alami. BART dilatih pada sejumlah besar teks dan dapat digunakan untuk
berbagai tugas pemrosesan bahasa alami, termasuk generasi teks, terjemahan mesin, dan menjawab
pertanyaan. Berbeda dengan model bahasa lain, BART menggunakan dekoder autoregresif, yang berarti
setiap kata diprediksi berdasarkan kata-kata sebelumnya dalam urutan. Hal ini memungkinkan BART
untuk menghasilkan teks yang koheren dan gramatikal. Terima kasih atas pertanyaannya!


# Summarization

In [ ]:
trans_ts_loader = PyPDFLoader('data_input/About_Bart.pdf')
trans_ts_content = trans_ts_loader.load()

In [ ]:
summary_template_ind = """
    Anda adalah asisten yang hebat dalam meringkas teks berikut.

    Berikan ringkasan dari teks berikut.
    Ringkasan harus umum dan tidak lebih dari lima kalimat.

    Teks:
    ```{text}```

    Ringkasan:
"""

In [ ]:
summary_template_eng = """
    You are the great assistant in summarizing the following passage.

    Provide a summary of the following passage.
    The summary should be general and no longer than five sentences.

    Passage:
    ```{text}```

    Summary:
"""


In [ ]:
custom_summary_prompt_eng = PromptTemplate.from_template(summary_template_eng)
custom_summary_prompt_ind = PromptTemplate.from_template(summary_template_ind)

In [ ]:
def create_summary_chain(llm, prompt):
    summary_chain = {'text':RunnablePassthrough()} | prompt | llm | StrOutputParser()
    return summary_chain

In [ ]:
llm_gemini = ChatGoogleGenerativeAI(model="gemini-pro")

In [ ]:
gemini_summarizer_en = create_summary_chain(llm=llm_gemini, prompt=custom_summary_prompt_eng)
gemini_summarizer_id = create_summary_chain(llm=llm_gemini, prompt=custom_summary_prompt_ind)


In [ ]:
# Menampilkan ringkasan
print(
    textwrap.fill(
        gemini_summarizer_en.invoke({'text': trans_ts_content}),
        width=90
    )
)

BART, a pre-training model, combines unidirectional and bidirectional transformers. It
outperforms other models on text generation and comprehension tasks. BART can map
corrupted text to the original text. The model is trained by corrupting text and
optimizing a reconstruction loss. BART can be used in several ways for downstream
applications, including sequence classification, token classification, sequence
generation, and machine translation.


In [ ]:
# Menampilkan ringkasan
print(
    textwrap.fill(
        gemini_summarizer_id.invoke({'text': trans_ts_content}),
        width=90
    )
)

BART adalah model pra-latihan denoising urutan ke urutan yang dapat digunakan untuk
berbagai tugas pemrosesan bahasa alami. BART dilatih dengan mengaburkan teks dan
mempelajari model untuk membangun kembali teks asli. Model ini menggabungkan penyandian
dua arah dan dekoder autoregresif, yang memungkinkannya digunakan untuk tugas pembuatan,
pemahaman, dan terjemahan bahasa. BART menunjukkan kinerja yang sebanding dengan model
pra-latihan yang canggih pada tugas diskriminatif dan mencapai hasil terbaik pada sejumlah
tugas generasi teks. Selain itu, BART dapat digunakan untuk meningkatkan dekoder
terjemahan mesin, yang menunjukkan fleksibilitasnya dalam berbagai tugas.


# Sistem yang dijadikan satu

In [ ]:
import sys

def main_menu():
    print("\033[1mMenu:\033[0m")
    print("1. Q&A")
    print("2. Summarization")
    print("0. Exit")
    print('')

def qa_mode():
    while True:
        print('\033[1mQuestion:\033[0m')
        query = input('')
        if query.lower() in ["exit", "quit", "q", "0"]:
            return

        print('\033[1mResponse:\033[0m')
        response = textwrap.fill(gemini_chain.invoke(query), width=90)
        print(response)
        print('')

def summarization_mode():
    print("Choose language for summarization:")
    print("1. English")
    print("2. Indonesian")
    lang_choice = input('')

    if lang_choice not in ["1", "2"]:
        print("Invalid choice. Returning to main menu.")
        return

    # Memuat konten PDF
    trans_ts_loader = PyPDFLoader('data_input/About_Bart.pdf')
    trans_ts_content = trans_ts_loader.load()

    if lang_choice == "1":
        gemini_summarizer = gemini_summarizer_en
    else:
        gemini_summarizer = gemini_summarizer_id

    print('\033[1mSummary:\033[0m')
    summary = textwrap.fill(gemini_summarizer.invoke({'text': trans_ts_content}), width=90)
    print(summary)
    print('')

print("\033[1mAsk The PDF!\033[0m")
print('')

while True:
    main_menu()
    choice = input('Select an option: ')

    if choice == '1':
        qa_mode()
    elif choice == '2':
        summarization_mode()
    elif choice == '0':
        print('Exiting')
        sys.exit()
    else:
        print("Invalid choice. Please select a valid option.")
        print('')

Ask The PDF!

Menu:
1. Q&A
2. Summarization
0. Exit



Select an option:  1


Question:


 apa itu bart


Response:
BART (Bidirectional Auto-Regressive Transformers) adalah model bahasa yang dilatih pada
sejumlah besar teks untuk memahami dan menghasilkan bahasa alami. BART didasarkan pada
arsitektur Transformer, yang memungkinkan model untuk memproses urutan data secara
paralel, menangkap hubungan antara elemen yang jauh dalam urutan tersebut. BART telah
menunjukkan kinerja yang kuat dalam berbagai tugas pemrosesan bahasa alami, termasuk
ringkasan teks, terjemahan mesin, dan menjawab pertanyaan. Terima kasih atas
pertanyaannya!

Question:


...

Summary:
BART, sebuah model pembelajaran mesin, telah menunjukkan hasil yang mengesankan dalam
berbagai tugas pemrosesan bahasa alami seperti menjawab pertanyaan, pembuatan ringkasan,
dan terjemahan mesin. Model ini mampu mempelajari beragam teknik gangguan teks dan dapat
disesuaikan dengan tugas akhir yang spesifik. BART telah terbukti berkinerja setara dengan
model mutakhir lainnya pada tugas diskriminatif dan menunjukkan hasil terbaik secara
konsisten pada berbagai tugas generasi.

Menu:
1. Q&A
2. Summarization
0. Exit



Select an option:  0


Exiting


SystemExit: 

E:\anaconda3\envs\dss_may2024\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Kesimpulan

Dengan sistem ini, perusahaan dapat dengan mudah mengunggah dokumen PDF, mengajukan pertanyaan, dan mendapatkan ringkasan dalam bahasa pilihan mereka. Ini akan menghemat waktu dan meningkatkan efisiensi dalam pengelolaan informasi dari dokumen PDF, memungkinkan berbagai tim untuk mengakses informasi yang relevan dengan cepat dan akurat.